![RP Tango](./Tango_Management_logo.png "RP Tango")

***
# Movie Time 1.0a
#### Web Scrap project to gather Cine Colombia movies on display and add IMDB Rating.

IMDB Rating Gathered from http://www.omdbapi.com/

***

In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import subprocess
import cfscrape

In [2]:
#Variables Globales
Cartelera = []
Cartelera.append(['Title', 'Year', 'Genre', 'imdbRating', 'Plot', 'Poster'])
MovieIndex=0
CarteleraExcel=pd.DataFrame({})

In [3]:
def getmovie(MovieName):
    """
    Function to retrieve IMDB Movie Information
    This Function is based on http://www.omdbapi.com/
    """
    APIkey="&apikey=67facf11" #use your unique API Key
    imdbQuery="http://www.omdbapi.com/?t="
    ApiCall_imdb=imdbQuery+MovieName+APIkey
    imdb_get = requests.get(ApiCall_imdb)
    movie_json = imdb_get.json()
    imdb_Error = "Movie not found!"
    if len(movie_json)>2 :
        imdb_Title = movie_json['Title']
        imdb_Year=movie_json['Year']
        imdb_Genre=movie_json['Genre']
        imdb_Rating=movie_json['imdbRating']
        imdb_plot=movie_json['Plot']
        MovieBanner=movie_json['Poster']
        allInfo=[]
        allInfo=[imdb_Title,imdb_Year,imdb_Genre,imdb_Rating,imdb_plot,MovieBanner]
        return allInfo
    else:
        return imdb_Error
#getmovie("Spider-Man: Far from Home")

In [4]:
urlpage =  'https://www.cinecolombia.com/cartelera/bogota'
scraper = cfscrape.create_scraper()
soup=BeautifulSoup(scraper.get(urlpage).content)
links=soup.find_all('h3',attrs={'class':'titulo_pelicula_typo01'}) # Obtengo Listado de todos los URL's

In [5]:
for link in links:    
    if len(link) == 0: 
        continue
    movielink=links[MovieIndex].find('a').get('href') # Recorro Links
    InnerSoup=BeautifulSoup(scraper.get(movielink).content) # Scrape URL Secundario
    OriginalName=InnerSoup.find(attrs={'class':'field field-name-field-titulo-original'}).getText()
    OriginalName = OriginalName.strip('Título original: ').strip('&nbsp;') #Obtengo Nombre Pelicula en Ingles
    FullDescription=getmovie(OriginalName) # Query a IMDB Api
    Cartelera.append(FullDescription)
    MovieIndex+=1
    
dataCartelera=pd.DataFrame(Cartelera)
dataCartelera.to_excel (r'./Cartelera.xlsx', header=False,index = False, sheet_name='CineColombia')

In [ ]:
## ---------- EJEMPLO  sin CFSCRAPER, CF DDOS detecta el intento de scrape y bloquea ----
##  <div class="cf-browser-verification cf-im-under-attack">
# urlpage =  'https://www.cinecolombia.com/cartelera/bogota'
# user_agent = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"}
# Ltable = requests.get(urlpage,timeout=6,allow_redirects=False,headers=user_agent)
# time.sleep(1)
# html_doc = Ltable.text
# soupfake = BeautifulSoup(html_doc)
# print(soupfake)